In [1]:
import ee
import geemap

# Authentifier et initialiser Google Earth Engine
ee.Authenticate()
ee.Initialize(project="project-2026-476714")


Enter verification code: 4/1Ab32j92DXVJiY9Xb2l0apBLTq_qHFINM_Sl3y2WuhyD9qKlp1bgTml64oLg

Successfully saved authorization token.


In [28]:
import ee
import geemap

In [29]:
import ee
ee.Initialize(project="project-2026-476714")

print("Earth Engine est connecté :", ee.Number(1).getInfo())


Earth Engine est connecté : 1


In [30]:
# ===== 2.1. Préfixe des assets =====
ASSET = "projects/project-2026-476714/assets/"

def fc(name):
    """Raccourci pour créer une FeatureCollection depuis ton projet."""
    return ee.FeatureCollection(ASSET + name)

# ===== 2.2. GADM : limites administratives du Tchad =====
tcd0 = fc("gadm41_TCD_0")  # pays
tcd1 = fc("gadm41_TCD_1")  # régions
tcd2 = fc("gadm41_TCD_2")  # départements
tcd3 = fc("gadm41_TCD_3")  # sous-préfectures

# ===== 2.3. OSM : couches principales =====
buildings = fc("gis_osm_buildings_a_free_1")
landuse   = fc("gis_osm_landuse_a_free_1")
natural_a = fc("gis_osm_natural_a_free_1")
natural_l = fc("gis_osm_natural_free_1")
places_a  = fc("gis_osm_places_a_free_1")
places_p  = fc("gis_osm_places_free_1")
pois_a    = fc("gis_osm_pois_a_free_1")
pois_p    = fc("gis_osm_pois_free_1")
pofw_a    = fc("gis_osm_pofw_a_free_1")
pofw_p    = fc("gis_osm_pofw_free_1")
railways  = fc("gis_osm_railways_free_1")
roads     = fc("gis_osm_roads_free_1")
traffic_a = fc("gis_osm_traffic_a_free_1")
traffic_l = fc("gis_osm_traffic_free_1")
transport_a = fc("gis_osm_transport_a_free_1")
transport_l = fc("gis_osm_transport_free_1")
waterways_l = fc("gis_osm_waterways_free_1")
waterways_a = fc("gis_osm_waterways_a_free_1")

print("Collections chargées.")


Collections chargées.


In [31]:
def add_counts(region):
    pts = collection.filterBounds(region.geometry())
    return region.set('mon_compte', pts.size())


In [52]:
import ee, geemap, os

# ==============================


# ==============================
# 1) Chemin de sortie
# ==============================
output_path = r"C:\Users\admin\Pictures\ISEP3\TP2\outputs"
os.makedirs(output_path, exist_ok=True)

# ==============================
# 2) Carte nationale du réseau routier
# ==============================
m = geemap.Map()
m.add_basemap('OpenStreetMap')       # fond de carte
m.center_object(tcd0, 5)             # centrage sur le Tchad

# Limites régionales
regions_style = {'color': 'blue', 'width': 2, 'fillColor': '00000000'}
m.addLayer(tcd1.style(**regions_style), {}, "Régions (GADM 1)")

# Routes classées
primary   = roads.filter(ee.Filter.eq('fclass', 'primary'))
secondary = roads.filter(ee.Filter.eq('fclass', 'secondary'))
tertiary  = roads.filter(ee.Filter.eq('fclass', 'tertiary'))

primary_style   = {'color': 'red',    'width': 4}
secondary_style = {'color': 'orange', 'width': 2.5}
tertiary_style  = {'color': 'yellow', 'width': 1.5}

m.addLayer(primary.style(**primary_style),   {}, "Routes primaires")
m.addLayer(secondary.style(**secondary_style), {}, "Routes secondaires")
m.addLayer(tertiary.style(**tertiary_style),  {}, "Routes tertiaires")

m.add_legend(
    title="Réseau routier du Tchad",
    legend_dict={
        "Régions": "blue",
        "Routes primaires": "red",
        "Routes secondaires": "orange",
        "Routes tertiaires": "yellow"
    }
)

# ==============================
# 3) Export HTML seulement
# ==============================
html_path = os.path.join(output_path, "carte_reseau_routier_tchad.html")
m.save(html_path)

print("Carte nationale enregistrée ici :")
print(html_path)

# Affichage dans le notebook
m


Carte nationale enregistrée ici :
C:\Users\admin\Pictures\ISEP3\TP2\outputs\carte_reseau_routier_tchad.html


Map(center=[15.306204623659646, 18.656507734823464], controls=(WidgetControl(options=['position', 'transparent…

In [51]:


# =====================================================
# 0) Définir le chemin d'enregistrement
# =====================================================
output_path = r"C:\Users\admin\Pictures\ISEP3\TP2\outputs"
os.makedirs(output_path, exist_ok=True)

# =====================================================
# 1) Définir la région à afficher
# =====================================================
nom_region = "Mayo-Kebbi Ouest"    # Tu peux changer pour "Mayo-Kebbi Est"

# Sélection de la région GADM
region_sel = tcd1.filter(ee.Filter.eq('NAME_1', nom_region))

# Filtrer les routes de cette région
roads_region = roads.filterBounds(region_sel.geometry())

# =====================================================
# 2) Préparer la carte
# =====================================================
m_reg = geemap.Map()
m_reg.add_basemap('OpenStreetMap')
m_reg.center_object(region_sel, 7)

# Style des régions
regions_style = {'color': 'blue', 'width': 2, 'fillColor': '00000000'}

m_reg.addLayer(
    region_sel.style(**regions_style),
    {},
    f"Région : {nom_region}"
)

# =====================================================
# 3) Routes classées
# =====================================================
primary_r   = roads_region.filter(ee.Filter.eq('fclass', 'primary'))
secondary_r = roads_region.filter(ee.Filter.eq('fclass', 'secondary'))
tertiary_r  = roads_region.filter(ee.Filter.eq('fclass', 'tertiary'))

primary_style   = {'color': 'red',    'width': 4}
secondary_style = {'color': 'orange', 'width': 2.5}
tertiary_style  = {'color': 'yellow', 'width': 1.5}

m_reg.addLayer(primary_r.style(**primary_style),   {}, "Primaires")
m_reg.addLayer(secondary_r.style(**secondary_style), {}, "Secondaires")
m_reg.addLayer(tertiary_r.style(**tertiary_style),  {}, "Tertiaires")

# =====================================================
# 4) Ajouter la légende
# =====================================================
m_reg.add_legend(
    title=f"Réseau routier - {nom_region}",
    legend_dict={
        "Routes primaires": "red",
        "Routes secondaires": "orange",
        "Routes tertiaires": "yellow",
        "Limites régionales": "blue"
    }
)

# =====================================================
# 5) EXPORT LOCAL (HTML + PNG)
# =====================================================

# --- HTML interactif ---
html_path = os.path.join(output_path, f"carte_routes_{nom_region}.html")
m_reg.save(html_path)



print("Export terminé avec succès !")
print("HTML :", html_path)


# Affichage dans le notebook
m_reg


Export terminé avec succès !
HTML : C:\Users\admin\Pictures\ISEP3\TP2\outputs\carte_routes_Mayo-Kebbi Ouest.html


Map(center=[9.334496062433697, 14.752695258864707], controls=(WidgetControl(options=['position', 'transparent_…

In [55]:
# ===== 4.1. Préparer les filtres POI =====

# Hypothèse : champ 'fclass' contient la catégorie OSM.
# Adapte les valeurs si besoin après avoir inspecté quelques features.

# Santé : pharmacies, hôpitaux, cliniques...
health_classes = ['pharmacy', 'hospital', 'clinic', 'doctors']

health_pois = pois_p.filter(ee.Filter.inList('fclass', health_classes))

# Éducation : écoles, collèges, universités...
school_classes = ['school', 'college', 'university']

school_pois = pois_p.filter(ee.Filter.inList('fclass', school_classes))

# Lieux de culte : directement dans pofw_p (points)
worship_pois = pofw_p  # déjà spécifique


# ===== 4.2. Fonction utilitaire pour compter les points par région =====

def add_point_count(regions_fc, points_fc, field_name):
    """
    Ajoute à chaque feature de regions_fc une propriété field_name
    contenant le nombre de points_fc dans la géométrie de la région.
    """
    def _count_points(region):
        pts_in_region = points_fc.filterBounds(region.geometry())
        return region.set(field_name, pts_in_region.size())
    return regions_fc.map(_count_points)


# ===== 4.3. Appliquer aux régions =====

regions_stats = tcd1

regions_stats = add_point_count(regions_stats, health_pois, 'nb_sante')
regions_stats = add_point_count(regions_stats, school_pois, 'nb_ecoles')
regions_stats = add_point_count(regions_stats, worship_pois, 'nb_culte')

# Vérifier une région en particulier (premier élément)
first_region = regions_stats.first()
print(first_region.toDictionary().getInfo())


{'CC_1': 'NA', 'COUNTRY': 'Chad', 'ENGTYPE_1': 'Region', 'GID_0': 'TCD', 'GID_1': 'TCD.23_1', 'HASC_1': 'TD.BI', 'ISO_1': 'NA', 'NAME_1': 'Wadi Fira', 'NL_NAME_1': 'NA', 'TYPE_1': 'Région', 'VARNAME_1': 'Biltine', 'nb_culte': 2, 'nb_ecoles': 3, 'nb_sante': 14}


In [58]:
import pandas as pd

# 1) Récupérer la FeatureCollection côté client (ATTENTION : à faire seulement
#    pour de petites collections, comme ici)
regions_info = regions_stats.getInfo()

# 2) Extraire la liste des "features"
features = regions_info['features']

# 3) Construire une liste de dictionnaires avec les propriétés uniquement
rows = [f['properties'] for f in features]

# 4) Créer le DataFrame pandas
df = pd.DataFrame(rows)

output_path = r"C:\Users\admin\Pictures\ISEP3\TP2\outputs"
os.makedirs(output_path, exist_ok=True)

# Chemins des fichiers
csv_path  = os.path.join(output_path, "services_sociaux_par_region.csv")
xlsx_path = os.path.join(output_path, "services_sociaux_par_region.xlsx")

# Sauvegarde
df.to_csv(csv_path, index=False)
df.to_excel(xlsx_path, index=False)

print("Fichiers enregistrés :")
print(csv_path)
print(xlsx_path)
# 5) Afficher seulement quelques colonnes intéressantes
df[['NAME_1', 'nb_sante', 'nb_ecoles', 'nb_culte']]


Fichiers enregistrés :
C:\Users\admin\Pictures\ISEP3\TP2\outputs\services_sociaux_par_region.csv
C:\Users\admin\Pictures\ISEP3\TP2\outputs\services_sociaux_par_region.xlsx


,NAME_1,nb_sante,nb_ecoles,nb_culte
0,Wadi Fira,14,3,2
1,Borkou,4,9,3
2,Guéra,4,14,4
3,Logone Occidental,5,7,7
4,Logone Oriental,14,14,13
5,Ville de N'Djamena,105,115,33
6,Barh el Ghazel,11,3,6
7,Batha,7,0,1
8,Ennedi Est,0,0,0
9,Ennedi Ouest,0,1,0


In [9]:
# Collection des aires protégées (asset GEE)
wdpa = ee.FeatureCollection("projects/project-2026-476714/assets/WDPA_TCD_polygons_merged")

# Régions administratives (niveau 1, GAUL simplifié)
regions = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level1") \
    .filter(ee.Filter.eq("ADM0_NAME", "Chad"))

# Géométrie du pays
tchad_geom = regions.geometry()


In [59]:
m = geemap.Map(center=[15, 19], zoom=5)

# Style des régions
style_regions = {
    "color": "1f4e79",       # bleu foncé
    "fillColor": "00000000", # transparent
    "width": 1.5
}

# Style des aires protégées (ROUGE)
style_wdpa = {
    "color": "b30000",        # rouge foncé (contour)
    "fillColor": "ff000050",  # rouge transparent
    "width": 1
}

# Ajout des calques
m.addLayer(regions.style(**style_regions), {}, "Régions")
m.addLayer(wdpa.style(**style_wdpa), {}, "Aires protégées (rouge)")
output_path = r"C:\Users\admin\Pictures\ISEP3\TP2\outputs"
os.makedirs(output_path, exist_ok=True)

html_path = os.path.join(output_path, "carte_aires_protegees_tchad.html")
m.save(html_path)

print("Carte HTML enregistrée ici :")
print(html_path)

m


Carte HTML enregistrée ici :
C:\Users\admin\Pictures\ISEP3\TP2\outputs\carte_aires_protegees_tchad.html


Map(center=[15, 19], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', tran…

In [60]:
# Fonction : ajoute le nombre d'AP intersectant chaque région
def add_count(feat):
    geom = feat.geometry()
    count = wdpa.filterBounds(geom).size()
    return feat.set("count", count)

# Appliquer la fonction à chaque région
wdpa_by_region = regions.map(add_count)

# Conversion en DataFrame
df_regions = geemap.ee_to_df(wdpa_by_region)

# Colonnes utiles
df_regions = df_regions[["ADM1_NAME", "count"]].sort_values("count", ascending=False)
# --------------------------------------------------------
# 3) Enregistrement local (CSV + Excel)
# --------------------------------------------------------
output_path = r"C:\Users\admin\Pictures\ISEP3\TP2\outputs"
os.makedirs(output_path, exist_ok=True)

csv_path  = os.path.join(output_path, "wdpa_par_region.csv")
xlsx_path = os.path.join(output_path, "wdpa_par_region.xlsx")

df_regions.to_csv(csv_path, index=False)
df_regions.to_excel(xlsx_path, index=False)
df_regions


,ADM1_NAME,count
5,Baguirmi,2
27,Salamat,1
26,Logone Occidental,1
25,Lac,1
20,Sila,1
11,Mont De Lam,1
15,Barh Koh,0
24,Guera,0
23,Biltine,0
22,Tandjile Ouest,0
